In [1]:
import urllib  
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

'Add these in when running by itself (not part of a loop)'

'Add these in when running by itself (not part of a loop)'

In [2]:
opener=urllib.request.build_opener()
opener.addheaders=[('User-Agent', 'Google Chrome/57.0.2987.133')]

In [3]:
url=('http://www.trademe.co.nz')
ourURL=opener.open(url).read()
import re

In [4]:
soup=BeautifulSoup(ourURL, "lxml")

In [5]:
#soup #this is all the HTML on the given URL.

#print(soup(text=re.compile("Mobile")))

In [6]:
def urlmaker(page):
    return('http://www.trademe.co.nz/browse/categorylistings.aspx?rptpath=344-422-&mcatpath=mobile-phones%2fmobile-phones&page='
           +str(page))
urlmaker(0)

'http://www.trademe.co.nz/browse/categorylistings.aspx?rptpath=344-422-&mcatpath=mobile-phones%2fmobile-phones&page=0'

In [9]:
def soupify(page):
    return(BeautifulSoup(opener.open(urlmaker(page)).read(),"lxml") )
#soupify(0)

In [10]:
#this is every instance of soupify, to make less reliance on functions later-> and to hault time.
#I will run this once per day.

soups=[]

for i in range(0,120):
        soups.append(soupify(i))

#soups[0]

#UPDATE: Yeah this made it work like 10X faster. :>

In [11]:
#definitions have to be more complex here...

#the below contains all items as a single string.

def item(page):
    p=soups[page].findAll('a', {'class':re.compile("tile")})
    paragraphs=[]
    
    for x in p:
        paragraphs.append(str(x))
    for i in range(len(paragraphs)):  
        paragraphs[i]=" ".join(paragraphs[i].split())
    return(paragraphs)

#item(0)

#This is information on each item for that page. If I can find an effecive way to
#subset this, I can gather information in 1 step

In [12]:
#TIME TO GO HYPERSPEED.

items=[]
for j in range(len(soups)):
        items.append(item(j))
     

In [13]:
def location(page):
    liste=[]
    for i in range(len(items[page])):
        liste.append(re.search('<div class="location">(.+?)</div>', 
                               items[page][i]).group(1)) 
    return(liste) 

def name(page):
    liste=[]
    for i in range(len(items[page])):
        liste.append(re.search('<div class="title">(.+?)</div>', 
                               items[page][i]).group(1))  
    return(liste)

def bidprices(page):
    liste=[]
    for i in range(len(items[page])):
        try:
            liste.append(re.search('<div class="listingBidPrice" id="SuperGridGallery_ctl00_BidInfo_listingBidPrice">(.+?)</div>'
                                   , items[page][i]).group(1))    
        except:
            liste.append('BuyNowOnly')   
    return(liste)

def buynowprices(page):
    liste=[]
    for i in range(len(items[page])):
        try:
            liste.append(re.search('<div class="listingBuyNowPrice" id="SuperGridGallery_ctl00_BuyNow_listingBuyNowPrice">(.+?)</div>'
                                   , items[page][i]).group(1))    
        except:
            liste.append('BidOnly')
    return(liste)


def buynowalt(page):
    liste=[]
    for i in range(len(items[page])):
        try:
            liste.append(re.search('<div class="listingBuyNowPrice with-was-price" id="SuperGridGallery_ctl00_BuyNow_listingBuyNowPrice">(.+?)</div>'
                                   , items[page][i]).group(1))    
        except:
            liste.append('NaN')
    return(liste)

def buynowalt2(page):
    liste=[]
    for i in range(len(items[page])):
        try:
            liste.append(re.search('div class="listingBuyNowPrice with-was-price with-was-price with-was-price" id="SuperGridGallery_ctl00_BuyNow_listingBuyNowPrice">(.+?)</div>'
                                   , items[page][i]).group(1))    
        except:
            liste.append('NaN')
    return(liste)

def ID(page):
    liste=[]
    for i in range(len(items[page])):
        liste.append(re.search('<div class="watchlist " id="watchlist(.+?)onclick', 
                               items[page][i]).group(1))  
    return(liste)


'Enter logic either side of (.+?) {in the red}'

'Enter logic either side of (.+?) {in the red}'

In [14]:
locations=[]

for i in range(len(soups)):
    locations.append(location(i))
    
flattened_locations = [val for sublist in locations for val in sublist]
#flattened_locations

In [15]:
names=[]

for i in range(len(soups)):
    names.append(name(i))
    
flattened_names = [val for sublist in names for val in sublist]
#flattened_names

In [16]:
bidpricess=[]

for i in range(len(soups)):
    bidpricess.append(bidprices(i))
    
flattened_bidprices = [val for sublist in bidpricess for val in sublist]
#flattened_bidprices

In [17]:
buynowpricess=[]

for i in range(len(soups)):
    buynowpricess.append(buynowprices(i))
    
flattened_buynowprices = [val for sublist in buynowpricess for val in sublist]
#flattened_buynowpricess

In [18]:
buynowpricessalt=[]

for i in range(len(soups)):
    buynowpricessalt.append(buynowalt(i))
    
flattened_buynowpricesalt = [val for sublist in buynowpricessalt for val in sublist]
#flattened_buynowpricess

In [19]:
buynowpricessalt2=[]

for i in range(len(soups)):
    buynowpricessalt2.append(buynowalt2(i))
    
flattened_buynowpricesalt2 = [val for sublist in buynowpricessalt2 for val in sublist]
#flattened_buynowpricess

In [20]:
IDS=[]

for i in range(len(soups)):
    IDS.append(ID(i))
    
flattened_IDS = [val for sublist in IDS for val in sublist]
#flattened_IDS

In [21]:
Frame=pd.DataFrame()

Frame['Location']=flattened_locations
Frame['Item_Name']=flattened_names
Frame['BidPrice']=flattened_bidprices
Frame['BuyNow_Price']=flattened_buynowprices
Frame['Price_Fixer']=flattened_buynowpricesalt
Frame['Price_Fixer2']=flattened_buynowpricesalt2
Frame['ID']=flattened_IDS

In [22]:
#Time to filtered

#would like to do this in one line...looks uggo af.
filtered=Frame[Frame.Item_Name.str.contains("Screen") == False]
filtered=filtered[filtered.Item_Name.str.contains("screen") == False]
filtered=filtered[filtered.Item_Name.str.contains("Case") == False]
filtered=filtered[filtered.Item_Name.str.contains("case") == False]
filtered=filtered[filtered.Item_Name.str.contains("Lense") == False]
filtered=filtered[filtered.Item_Name.str.contains("lense") == False]
filtered=filtered[filtered.Item_Name.str.contains("Trade") == False]
filtered=filtered[filtered.Item_Name.str.contains("trade") == False]
filtered=filtered[filtered.Item_Name.str.contains("adapter") == False]
filtered=filtered[filtered.Item_Name.str.contains("Adapter") == False]
filtered=filtered[filtered.Item_Name.str.contains("Dock") == False]
filtered=filtered[filtered.Item_Name.str.contains("dock") == False]
filtered=filtered[filtered.Item_Name.str.contains("parts") == False]
filtered=filtered[filtered.Item_Name.str.contains("Parts") == False]

print(len(filtered))

#List to use if doable in a one liner

to_remove=["Screen","screen","Case","case","Lense","lense","Trade","trade",
          "adapter","Adapter","Dock","dock","parts","Parts"]

#filtered

5937


In [23]:
lister=[]

for i in range(len(filtered)):
    if filtered.iloc[i]['BidPrice']=='BuyNowOnly' and filtered.iloc[i]['BuyNow_Price']=='BidOnly':
        lister.append(filtered.iloc[i]['Price_Fixer'])
    else:
         lister.append(filtered.iloc[i]['BuyNow_Price'])
#lister   

In [24]:
#del filtered['BuyNow_Price']
#del filtered['Price_Fixer']
filtered['BuyNowPrice']=lister
filtered

lister2=[]

for i in range(len(filtered)):
    if filtered.iloc[i]['BuyNowPrice']=='NaN':
        lister2.append(filtered.iloc[i]['Price_Fixer2'])
    else:
         lister2.append(filtered.iloc[i]['BuyNowPrice'])
            
filtered['BuyNowPricee']=lister2
#lister   

del filtered['BuyNow_Price']
del filtered['Price_Fixer']
del filtered['BuyNowPrice']
del filtered['Price_Fixer2']

In [25]:
import time
timestr = time.strftime("%Y+%m+%d-%H+%M+%S")
timestr

'2017+05+25-14+41+58'

In [26]:
filtered=filtered[filtered.BuyNowPricee.str.contains("NaN") == False]
print(len(filtered))
filtered2=filtered.drop_duplicates()  #duplicate rows for some reason.
print(len(filtered2))

path='D:\\TradeMe2\\'

df_obj = filtered2.select_dtypes(['object'])
filtered2[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

filtered2.to_csv(path+'filtered_data'+timestr+'.csv', dwdelimiter=',', encoding='utf-8')

5742
5649
